In [ ]:
# Install if needed
!pip install pandas matplotlib seaborn

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# Correct paths based on your folder structure
header_path = '/content/drive/My Drive/Colab Notebooks/claims-data/STONYBRK_20240531_HEADER.csv'
line_path = '/content/drive/My Drive/Colab Notebooks/claims-data/STONYBRK_20240531_LINE.csv'
code_path = '/content/drive/My Drive/Colab Notebooks/claims-data/STONYBRK_20240531_CODE.csv'

# Load data
import pandas as pd
df_header = pd.read_csv(header_path)
df_line = pd.read_csv(line_path)
df_code = pd.read_csv(code_path)

# Confirm shapes
print("HEADER:", df_header.shape)
print("LINE:", df_line.shape)
print("CODE:", df_code.shape)





In [ ]:

import pandas as pd

df_header = pd.read_csv(header_path)
df_line = pd.read_csv(line_path)
df_code = pd.read_csv(code_path)
df_header.head()
df_line.head()
df_code.head()


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_code)

In [ ]:

# HEADER

print(df_header.shape)
print(df_header.head())
print(df_header.info())
print(df_header.isnull().sum())
print(df_header.describe())


In [ ]:

df_header['ServiceFromDate'] = pd.to_datetime(df_header['ServiceFromDate'])
df_header['ServiceToDate'] = pd.to_datetime(df_header['ServiceToDate'])



In [ ]:

# Explore LINE file
print("LINE shape:", df_line.shape)
print(df_line.head())        # First 5 rows
print(df_line.info())        # Column names and data types
print(df_line.isnull().sum())# Missing values per column
print(df_line.describe())    # Basic stats for numeric columns


In [ ]:

# Explore CODE file
print("CODE shape:", df_code.shape)
print(df_code.head())        # First 5 rows
print(df_code.info())        # Column names and data types
print(df_code.isnull().sum())# Missing values per column
print(df_code.describe())    # Basic stats for numeric columns


In [ ]:

unique_claims = df_header['ProspectiveClaimId'].nunique()
print("Unique claims:", unique_claims)


In [ ]:

avg_lines = df_line.groupby('ProspectiveClaimId').size().mean()
print("Avg service lines per claim:", avg_lines)

avg_dx = df_code.groupby('ProspectiveClaimId').size().mean()
print("Avg diagnosis codes per claim:", avg_dx)



In [ ]:
print("Date range:", df_header['ServiceFromDate'].min(), "to", df_header['ServiceToDate'].max())


# Part 1: Data Loading & Exploration

## Header File Exploration
- Contains high-level claim information such as provider NPIs, service dates, and payer details.
- Observed columns: ProspectiveClaimId, ServiceFromDate, ServiceToDate, PrimaryPayerName, PlaceOfService, etc.
- Missing values: Minimal (mostly zero for some columns like WorkQueName).
- Total rows: 388 claims.

## LINE File Exploration
- Contains service line details including HCPCS codes, modifiers, charges, and units.
- Observed columns: ProspectiveClaimId, HCPCS, Charges, Units, DxMapDelim.
- Missing values: Few or none in critical columns.
- Average service lines per claim: **1.34**

## CODE File Exploration
- Contains diagnosis codes (ICD-10) linked to claims.
- Observed columns: ProspectiveClaimId, CodeValue, CodeQualifier.
- Missing values: None in key columns.
- Average diagnosis codes per claim: **3.96**

## Key Metrics
- **Unique claims:** 388
- **Date range:** 2023-09-25 to 2024-05-29
- **Avg service lines per claim:** 1.34
- **Avg diagnosis codes per claim:** 3.96




## PART 2
### Q1: Top 5 Billing Providers
- The top 5 billing providers were identified based on claim volume.
- Visualization shows provider NPIs and their respective claim counts.


In [ ]:

# Top 5 billing providers by number of claims
top_providers = df_header['BillingProviderNPI'].value_counts().head(5)
print(top_providers)

# Bar chart
plt.figure(figsize=(8,5))
sns.barplot(x=top_providers.index, y=top_providers.values, palette="Blues_d")
plt.title("Top 5 Billing Providers by Number of Claims")
plt.xlabel("Billing Provider NPI")
plt.ylabel("Number of Claims")
plt.show()



### Q2: Payer Mix Analysis
- The top 5 payers account for the majority of claims.
- Pie chart shows their percentage distribution.


In [ ]:

# Top 5 primary payers
payer_counts = df_header['PrimaryPayerName'].value_counts().head(5)
payer_percent = (payer_counts / len(df_header)) * 100
print(payer_counts)
print(payer_percent)

# Pie chart
plt.figure(figsize=(6,6))
plt.pie(payer_counts, labels=payer_counts.index, autopct='%1.1f%%', startangle=140)
plt.title("Top 5 Primary Payers by Claim Volume")
plt.show()



### Q3: Common Diagnoses
- The most frequent ICD-10 codes were identified.
- These represent common conditions in the dataset.


In [ ]:

top_dx = df_code['CodeValue'].value_counts().head(10)
print(top_dx)

# Bar chart
plt.figure(figsize=(10,5))
sns.barplot(x=top_dx.index, y=top_dx.values, palette="Greens_d")
plt.title("Top 10 Diagnosis Codes")
plt.xlabel("ICD-10 Code")
plt.ylabel("Frequency")
plt.xticks(rotation=45)
plt.show()



### Q4: Common Procedures
- The most frequent HCPCS codes were identified.
- Visualization shows top 10 procedures billed.


In [ ]:

top_proc = df_line['HCPCS'].value_counts().head(10)
print(top_proc)

# Bar chart
plt.figure(figsize=(10,5))
sns.barplot(x=top_proc.index, y=top_proc.values, palette="Purples_d")
plt.title("Top 10 Procedure Codes")
plt.xlabel("HCPCS Code")
plt.ylabel("Frequency")
plt.xticks(rotation=45)
plt.show()



### Q5: Place of Service Analysis
- Most claims were submitted for [insert top location].
- Chart shows distribution across service locations.


In [ ]:

pos_counts = df_header['PlaceOfService'].value_counts()
pos_percent = (pos_counts / len(df_header)) * 100
print(pos_counts)
print(pos_percent)

# Bar chart
plt.figure(figsize=(8,5))
sns.barplot(x=pos_counts.index, y=pos_counts.values, palette="Oranges_d")
plt.title("Claims by Place of Service")
plt.xlabel("Place of Service")
plt.ylabel("Number of Claims")
plt.xticks(rotation=45)
plt.show()


## PART 3

### Q6: Claims with High Service Line Counts
- Merged HEADER and LINE files to calculate service line counts and total charges.
- Identified claims with 5 or more service lines.
- These claims may represent complex cases or bundled services.


In [ ]:

# Merge HEADER and LINE files
merged_hl = df_header.merge(df_line, on='ProspectiveClaimId', how='inner')

# Calculate number of service lines and total charges per claim
line_summary = merged_hl.groupby('ProspectiveClaimId').agg(
    num_lines=('LinePos', 'count'),
    total_charges=('Charges', 'sum'),
    billing_provider=('BillingProviderNPI', 'first')
).reset_index()

# Filter claims with 5 or more service lines
high_line_claims = line_summary[line_summary['num_lines'] >= 5]
print(high_line_claims.head())



### Q7: Diagnosis-Procedure Combinations
- Linked claims to both procedures and diagnoses.
- For CPT code 99291 (critical care), the most common associated diagnoses were identified.


In [ ]:

# Merge all three files
merged_all = df_line.merge(df_code, on='ProspectiveClaimId').merge(df_header, on='ProspectiveClaimId')

# Filter for CPT code 99291
cpt_99291_dx = merged_all[merged_all['HCPCS'] == '99291']['CodeValue'].value_counts().head(10)
print(cpt_99291_dx)



### Q8: Charges by Payer
- Calculated total and average charges per claim for each payer.
- Top 10 payers by total charges are shown in the bar chart.


In [ ]:

# Merge HEADER and LINE files
merged_hl = df_header.merge(df_line, on='ProspectiveClaimId', how='inner')

# Group by PrimaryPayerName
charges_by_payer = merged_hl.groupby('PrimaryPayerName').agg(
    total_charges=('Charges', 'sum'),
    num_claims=('ProspectiveClaimId', 'nunique')
).reset_index()

# Calculate average charges per claim
charges_by_payer['avg_charge'] = charges_by_payer['total_charges'] / charges_by_payer['num_claims']

# Sort by total charges
charges_by_payer = charges_by_payer.sort_values(by='total_charges', ascending=False).head(10)
print(charges_by_payer)

# Visualization
plt.figure(figsize=(10,6))
sns.barplot(x='PrimaryPayerName', y='total_charges', data=charges_by_payer, palette="Blues_d")
plt.title("Top 10 Payers by Total Charges")
plt.xticks(rotation=45)
plt.ylabel("Total Charges")
plt.show()



# Merge HEADER and CODE files

## Part 4: Creative Analysis
### Question: Which providers bill for the most complex cases (highest number of diagnosis codes per claim)?
- Merged HEADER and CODE files to link providers with diagnosis codes.
- Calculated average number of diagnosis codes per claim for each provider.
- Top 10 providers with the highest complexity are shown in the bar chart.
- Insight: Providers with higher average diagnosis codes may handle more complex cases or multi-condition patients.


In [ ]:
merged_hc = df_header.merge(df_code, on='ProspectiveClaimId', how='inner')
# Count diagnosis codes per claim
dx_counts = merged_hc.groupby('ProspectiveClaimId').agg(
    num_dx=('CodeValue', 'count'),
    billing_provider=('BillingProviderNPI', 'first')
).reset_index()

# Aggregate by provider: average number of diagnosis codes per claim
provider_complexity = dx_counts.groupby('billing_provider').agg(
    avg_dx_per_claim=('num_dx', 'mean'),
    total_claims=('ProspectiveClaimId', 'count')
).reset_index()

# Sort by avg_dx_per_claim
provider_complexity = provider_complexity.sort_values(by='avg_dx_per_claim', ascending=False).head(10)

# Bar chart for top 10 providers by complexity
plt.figure(figsize=(10,6))
sns.barplot(x='billing_provider', y='avg_dx_per_claim', data=provider_complexity, palette="coolwarm")
plt.title("Top 10 Providers by Average Diagnosis Codes per Claim")
plt.xlabel("Billing Provider NPI")
plt.ylabel("Avg Diagnosis Codes per Claim")
plt.xticks(rotation=45)
plt.show()

